In [1]:
import pandas as pd
import numpy as np
import pathlib


In [2]:
path_data = pathlib.Path().cwd() / 'data'
path_raw = path_data / 'raw'

In [3]:
# Read CSV file with polars
df = pd.read_csv(path_raw / "dow_jones_close_prices_aug_dec_2024.csv")


In [4]:
## criar matriz de precos com numpy
prices_array = df.drop(columns='Date').to_numpy()

In [5]:
## gerar matriz de variação de preços diaria
## subtrair a matriz dela mesma com um deslocamento de 1 dia (row)
r = (prices_array[1:] / prices_array[:-1]) - 1

In [ ]:
# DIAGNOSTICS: Check shape and values of returns matrix
print(f"Returns matrix shape: {r.shape}")
print(f"Mean returns per asset:\n{r.mean(axis=0)}")
print(f"\nFirst few rows of returns matrix:\n{r[:5, :5]}")

In [6]:
# df.column.pct_change

In [ ]:
sigma = np.cov(r, rowvar=False)  # rowvar=False specifies that variables are in columns

In [ ]:
# DIAGNOSTICS: Check shape and values of covariance matrix
print(f"Covariance matrix shape: {sigma.shape}")
print(f"\nDiagonal of covariance matrix (variances):\n{np.diagonal(sigma)[:5]}")
print(f"\nCheck if matrix is symmetric (should be True): {np.allclose(sigma, sigma.T)}")

In [ ]:
## O problema é encontrar o argmax de para a função: f(w) = (r * w) / sqrt(wT * sigma * w )

def sharpe_ratio_simplified(w, r, sigma):
    # Calculate portfolio expected return
    port_return = np.dot(r, w).mean()
    
    # Calculate portfolio volatility
    port_vol = np.sqrt(np.dot(np.dot(w.T, sigma), w))
    
    return port_return / port_vol


In [9]:
## agora precisamos de uma forma de montar uma submatriz do r e sigma a partir dos indices das ações escolhidas

def get_submatrix_r(r, indices):
    return r[:, indices]

def get_submatrix_sigma(sigma, indices):
    return sigma[np.ix_(indices, indices)]

In [ ]:
from functools import partial

def gerar_vetores_aleatorios(n):
    return np.random.uniform(0, 0.2, n) ## impura

def gerar_w_valido(n: int = 25, max_tentativas: int = 1000):
    """
    Gera um vetor w válido, ou seja, que soma 1 e tem n elementos e nenhum ativo tem mais de 20% da carteira
    """
    gerar_vetores = partial(gerar_vetores_aleatorios, n)
    for _ in range(max_tentativas):
        valores = gerar_vetores() ## impura

        soma_atual = np.sum(valores)    
        valores_normalizados = valores * (1 / soma_atual)
        
        # Verificar se após normalização todos os valores ainda estão abaixo de 0.2
        if np.all(valores_normalizados <= 0.2):
            return valores_normalizados


In [ ]:
# TEST: Check if weights sum to 1
test_w = gerar_w_valido(30)
print(f"Sum of weights: {np.sum(test_w):.10f} (should be 1.0)")
print(f"Maximum weight: {np.max(test_w):.10f} (should be <= 0.2)")

In [ ]:
## teste: selecionar 25 das 30 ações aleatoriamente e usar um vetor w gerado aleatoriamente para calcular o sharpe ratio

def calcular_sharpe_ratio_teste(r, sigma, n=25):
    # Selecionar 25 ações aleatoriamente
    indices = np.random.choice(r.shape[1], size=n, replace=False)
    
    # Obter submatrizes
    r_sub = get_submatrix_r(r, indices)
    sigma_sub = get_submatrix_sigma(sigma, indices)
    
    # Gerar vetor w válido
    w = gerar_w_valido(n)
    
    # Calcular o Sharpe Ratio
    sr = sharpe_ratio_simplified(w, r_sub, sigma_sub)
    
    return sr, w, indices

sr, w_teste, indices_teste = calcular_sharpe_ratio_teste(r, sigma)

In [ ]:
# DIAGNOSTICS: Examine the sharpe ratio calculation in detail
# Get test data
r_sub = get_submatrix_r(r, indices_teste)
sigma_sub = get_submatrix_sigma(sigma, indices_teste)

# Calculate portfolio return
port_return = np.dot(r_sub, w_teste).mean()
# Calculate daily volatility
port_vol = np.sqrt(np.dot(np.dot(w_teste.T, sigma_sub), w_teste))

# Calculate daily and annualized Sharpe ratio
daily_sharpe = port_return / port_vol
annual_sharpe = (port_return * 252) / (port_vol * np.sqrt(252))

print(f"Portfolio expected daily return: {port_return:.6f}")
print(f"Portfolio daily volatility: {port_vol:.6f}")
print(f"Daily Sharpe ratio: {daily_sharpe:.6f}")
print(f"Annualized Sharpe ratio (252 trading days): {annual_sharpe:.6f}")
print(f"Simplified annualized Sharpe (using factor 252/sqrt(252)): {daily_sharpe * (252/np.sqrt(252)):.6f}")
print(f"Original test Sharpe: {sr:.6f}")

In [ ]:
from functools import partial

def calcular_n_sharpes_da_carteira(indices_carteira, r, sigma, n=1000):
    """
    Calcula n Sharpe Ratios para uma carteira com os indices passados
    """
    r_sub = get_submatrix_r(r, indices_carteira) ## pura
    sigma_sub = get_submatrix_sigma(sigma, indices_carteira) ## pura
    calculate_sr_with_w = partial(sharpe_ratio_simplified, r=r_sub, sigma=sigma_sub)
    ## Create a list of n random w vectors and then use map to calculate the sharpe ratio for each
    sharpes = list(map(calculate_sr_with_w, [gerar_w_valido(len(indices_carteira)) for _ in range(n)])) ## impura devido ao uso de gerar_w_valido
    
    return max(sharpes)

In [15]:
from multiprocessing import Pool
## sample teste
indices_teste = [np.random.choice(r.shape[1], size=25, replace=False) for _ in range(1000)]
calcular_n_sharpes_dos_indices = partial(calcular_n_sharpes_da_carteira, r=r, sigma=sigma)
result = Pool().map(calcular_n_sharpes_dos_indices, indices_teste)

In [21]:
# from functools import map
result_sem_paralelismo = list(map(calcular_n_sharpes_dos_indices, indices_teste))

In [17]:
max(result)

0.729057445403259

In [ ]:
# DIAGNOSTICS: Check range of Sharpe ratios
print(f"Min Sharpe ratio: {min(result):.6f}")
print(f"Max Sharpe ratio: {max(result):.6f}")
print(f"Mean Sharpe ratio: {np.mean(result):.6f}")
print(f"Sharpe ratio distribution: {np.percentile(result, [25, 50, 75, 90, 95, 99])}")

In [ ]:
# IMPLEMENTATION WITH ANNUALIZATION
def improved_sharpe_ratio(w, r, sigma, annualize=True):
    """Improved Sharpe ratio calculation with proper annualization"""
    # Calculate portfolio expected return
    port_return = np.dot(r, w).mean()
    
    # Calculate portfolio volatility
    port_vol = np.sqrt(np.dot(np.dot(w.T, sigma), w))
    
    # Calculate daily Sharpe ratio
    daily_sharpe = port_return / port_vol
    
    if annualize:
        # Properly annualize (assuming 252 trading days)
        return (port_return * 252) / (port_vol * np.sqrt(252))
    else:
        return daily_sharpe

# Test with the same data
improved_sr = improved_sharpe_ratio(w_teste, r_sub, sigma_sub, annualize=True)
print(f"Improved annualized Sharpe ratio: {improved_sr:.6f}")